In [1]:
# !pip install kaggle;

In [2]:
# import os
# os.environ['KAGGLE_CONFIG_DIR']='/content'

In [3]:
# !kaggle datasets download -d vipoooool/new-plant-diseases-dataset

Dataset URL: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset
License(s): copyright-authors
100% 2.70G/2.70G [00:29<00:00, 98.6MB/s]
100% 2.70G/2.70G [00:29<00:00, 97.5MB/s]


In [4]:
# !chmod 600 /content/kaggle.json

In [ ]:
# !unzip new-plant-diseases-dataset.zip;

In [1]:
# Intialization of Program. by Importing various LIbraries
import numpy as np
import matplotlib.pyplot as plt
# here we are working on Tensorflow version 2.1.0 so we need to write tensorflow.keras.
# keras is in built function in Tensorflow.
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

2024-05-04 21:22:41.068991: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-04 21:22:41.071164: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-04 21:22:41.173906: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-04 21:22:41.596456: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-04 21:22:43.670748: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [3]:
# Get the path to your train directory
train_dir = "../../train/"
valid_dir = "../../valid/"

img_size = 250
batch_size = 64

In [4]:
import cv2
import numpy as np
from skimage import exposure

In [5]:
def remove_bg(main_img):
    # Preprocessing
    img = cv2.cvtColor(main_img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (250, 250))

    gs = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Applying CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = exposure.equalize_adapthist(
        gs, kernel_size=None, clip_limit=0.015, nbins=256)

    # Convert the float64 image to 8-bit
    clahe_uint8 = (clahe * 255).astype(np.uint8)

    # Adjusting contrast in the enhanced image
    contrasted_image = np.clip(clahe_uint8, 0, 255)

    blur = cv2.GaussianBlur(contrasted_image, (35, 35), 0)

    return_otsu_threshould, im_bw_otsu = cv2.threshold(
        blur, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    kernel = np.ones((35, 35), np.uint8)
    closing = cv2.morphologyEx(im_bw_otsu, cv2.MORPH_CLOSE, kernel)
    # Contour
    contours, hierarchy = cv2.findContours(
        closing, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Finding the correct leaf contour from the list of contours

    def find_contour(cnts):
        contains = []
        y_ri, x_ri, _ = img.shape
        for cc in cnts:
            yn = cv2.pointPolygonTest(cc, (x_ri//2, y_ri//2), False)
            contains.append(yn)
            val = [contains.index(temp) for temp in contains if temp > 0]
            if len(val) < 1:
                return -1
        return val[0]

    index = find_contour(contours)

    if index == -1:
        return None

    cnt = contours[index]
    # Create a black image to draw contours
    black_img = np.zeros_like(img)
    # Draw contour on the black image
    mask = cv2.drawContours(black_img, [cnt], 0, (255, 255, 255), -1)
    img = cv2.bitwise_and(img, mask)

    return img

This is used to remove the background from images. ( We need to do it only once. )

In [6]:
import os


def generate_DS(dir):
    # Loop through subdirectories (classes)
    for class_dir in os.listdir(dir):
        class_path = os.path.join(dir, class_dir)

        if True:
            selected_images = os.listdir(class_path)

            for image in selected_images:
                image_path = os.path.join(class_path, image)
                img = cv2.imread(image_path)
                result = remove_bg(img)
                # print(result)
                if result is not None:
                    cv2.imwrite(image_path, result)


generate_DS(train_dir)
generate_DS(valid_dir)

In [7]:
# Use the modified train directory with `ImageDataGenerator`
datagen_train = ImageDataGenerator(horizontal_flip=True)
train_generator = datagen_train.flow_from_directory(train_dir,
                                                    target_size=(
                                                        img_size, img_size),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

Found 70295 images belonging to 38 classes.


In [8]:
# Use the modified train directory with `ImageDataGenerator`
datagen_test = ImageDataGenerator(horizontal_flip=True)
validation_generator = datagen_train.flow_from_directory(valid_dir,
                                                         target_size=(
                                                             img_size, img_size),
                                                         batch_size=batch_size,
                                                         class_mode='categorical',
                                                         shuffle=True)

Found 17572 images belonging to 38 classes.


In [9]:
detection = Sequential()

In [10]:
# convolutional layer-1
detection.add(Conv2D(64, (3, 3), padding='same', input_shape=(250, 250, 3)))
detection.add(BatchNormalization())
detection.add(Activation('relu'))
detection.add(MaxPooling2D(pool_size=(2, 2)))
detection.add(Dropout(0.25))

# 2 -convolutional layer-2
detection.add(Conv2D(128, (5, 5), padding='same'))
detection.add(BatchNormalization())
detection.add(Activation('relu'))
detection.add(MaxPooling2D(pool_size=(2, 2)))
detection.add(Dropout(0.25))

# 3 -convolutional layer-3
detection.add(Conv2D(256, (3, 3), padding='same'))
detection.add(BatchNormalization())
detection.add(Activation('relu'))
detection.add(MaxPooling2D(pool_size=(2, 2)))
detection.add(Dropout(0.25))

# 4 -convolutional layer-4
detection.add(Conv2D(512, (3, 3), padding='same'))
detection.add(BatchNormalization())
detection.add(Activation('relu'))
detection.add(MaxPooling2D(pool_size=(2, 2)))
detection.add(Dropout(0.25))

# 5 -convolutional layer-5
detection.add(Conv2D(512, (3, 3), padding='same'))
detection.add(BatchNormalization())
detection.add(Activation('relu'))
detection.add(MaxPooling2D(pool_size=(2, 2)))
detection.add(Dropout(0.25))

detection.add(Flatten())
detection.add(Dense(256))
detection.add(BatchNormalization())
detection.add(Activation('relu'))
detection.add(Dropout(0.25))

detection.add(Dense(512))
detection.add(BatchNormalization())
detection.add(Activation('relu'))
detection.add(Dropout(0.25))

detection.add(Dense(38, activation='softmax'))
optimum = Adam(learning_rate=0.005)

detection.compile(optimizer=optimum,
                  loss='categorical_crossentropy', metrics=['accuracy'])

/home/kunal/minorProject/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [11]:
detection.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 250, 250, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 250, 250, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 250, 250, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 125, 125, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 125, 125, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 128)  │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 125, 125, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 125, 125, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 62, 62, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 62, 62, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 62, 62, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 62, 62, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 31, 31, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 31, 31, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 31, 31, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 31, 31, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 31, 31, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 15, 15, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 15, 15, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 15, 15, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 15, 15, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 10,624,678 (40.53 MB)

 Trainable params: 10,620,198 (40.51 MB)

 Non-trainable params: 4,480 (17.50 KB)

In [12]:
print(train_generator.n)
print(validation_generator.n)

70295
17572


In [13]:
ephocs = 10
steps_per_epoch = train_generator.n//train_generator.batch_size

validation_steps = validation_generator.n//validation_generator.batch_size

history = detection.fit(x=train_generator,
                        steps_per_epoch=steps_per_epoch,
                        epochs=ephocs,
                        validation_data=validation_generator,
                        validation_steps=validation_steps)
detection.save('model.h5')

Epoch 1/10


/home/kunal/minorProject/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  26/1098 ━━━━━━━━━━━━━━━━━━━━ 6:14:09 21s/step - accuracy: 0.0546 - loss: 4.1045

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

# Get training and validation loss from the history object
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Get training and validation accuracy from the history object
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Plotting the training and validation loss
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(train_loss) + 1), train_loss, label='Training Loss')
plt.plot(range(1, len(val_loss) + 1), val_loss, label='Validation Loss')
plt.title('Loss vs. Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plotting the training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(range(1, len(train_acc) + 1), train_acc, label='Training Accuracy')
plt.plot(range(1, len(val_acc) + 1), val_acc, label='Validation Accuracy')
plt.title('Accuracy vs. Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()